In [73]:
import numpy as np
import math as m
import scipy as sc
import pandas as pd
from sklearn.datasets import make_blobs
import plotly.express as px

import logging
import sys

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

In [74]:
COLORS = [
    "#f7dc6f",
    "#82e0aa",
    "#f1948a",
    "#499cef",
    "#f5b041",
    "#a569bd",
    "#e74c3c",
    "#2ecc71",
    "#3498db",
    "#e67e22",
    "#9b59b6",
    "#1abc9c",
    "#34495e",
    "#d35400",
    "#c0392b",
    "#16a085",
    "#2980b9",
    "#8e44ad",
]

def twospirals(n_points, noise=.5):
    n = np.sqrt(np.random.rand(n_points,1)) * 780 * (2*np.pi)/360
    d1x = -np.cos(n)*n + np.random.rand(n_points,1) * noise
    d1y = np.sin(n)*n + np.random.rand(n_points,1) * noise
    return (np.vstack((np.hstack((d1x,d1y)),np.hstack((-d1x,-d1y)))), np.hstack((np.zeros(n_points, dtype=int),np.ones(n_points, dtype=int))))

def plot_dataframe(df, title='', x='x', y='y', label='label'):
    df[label] = df[label].astype(str)
    fig = px.scatter(
        df, 
        x=x, 
        y=y, 
        symbol=label,
        color=label,
        color_discrete_sequence= COLORS, 
        title=title
    )
    fig.update_traces(marker=dict(size=5, symbol='circle'))

    return fig

def plot_centroid(fig, centroid, colors = ['#a569bd'], marker_size=10, symbol = 'star', name = 'Centroid'):
    fig.add_scatter(
        x=[centroid[0]], 
        y=[centroid[1]], 
        mode='markers',
        marker=dict(
            size = marker_size,
            color = colors,
            symbol = symbol,
        ), 
        name=name
    )
    return fig

def create_constraints(labels, probability=0.01, seed=0):
    n_points = len(labels)
    constraints = np.zeros((n_points, n_points), dtype=int)
    state = np.random.RandomState(seed=seed)
    for i in range(n_points):
        for j in range(i +1, n_points):
            if state.rand() < probability:
                if labels[i] == labels[j]:
                    constraints[i, j] = 1
                    constraints[j, i] = 1
                elif labels[i] != labels[j]:
                    constraints[i, j] = -1
                    constraints[j, i] = -1
    return constraints

In [75]:
X, y = make_blobs(n_samples=300, centers=10, cluster_std=1.0, random_state=42)
constraints = create_constraints(y, probability=0.01, seed=42)
# df = pd.DataFrame(X, columns=['x', 'y'])
# df['label'] = y
# fig = plot_dataframe(df, title='Data with Constraints', x='x', y='y', label='label')
# fig = plot_centroid(fig, np.mean(X, axis=0), colors=['#a569bd'], marker_size=10, symbol='star', name='Centroid')
# fig.show()

In [79]:
from clustlib.kmean.rdpmean import RDPM
from clustlib.utils.simpleconstraints import SimpleConstraints

rdpm = RDPM(constraints=SimpleConstraints(constraints), n_clusters = 4, max_iter = 1000, tol = 1e-10, limit=1.5, rate = 1.2)
rdpm.fit(X)


2025-05-31 17:02:37,061 - DEBUG - Fitting RDPM model
2025-05-31 17:02:37,062 - DEBUG - Delta is None, convergence cannot be checked.
2025-05-31 17:02:37,065 - DEBUG - Instance 1 exceeds limit, creating new cluster
2025-05-31 17:02:37,067 - DEBUG - Instance 2 exceeds limit, creating new cluster
2025-05-31 17:02:37,069 - DEBUG - Instance 3 exceeds limit, creating new cluster
2025-05-31 17:02:37,071 - DEBUG - Instance 4 exceeds limit, creating new cluster
2025-05-31 17:02:37,073 - DEBUG - Instance 5 exceeds limit, creating new cluster
2025-05-31 17:02:37,075 - DEBUG - Instance 6 exceeds limit, creating new cluster
2025-05-31 17:02:37,077 - DEBUG - Instance 7 exceeds limit, creating new cluster
2025-05-31 17:02:37,081 - DEBUG - Instance 10 exceeds limit, creating new cluster
2025-05-31 17:02:37,085 - DEBUG - Instance 13 exceeds limit, creating new cluster
2025-05-31 17:02:37,087 - DEBUG - Instance 14 exceeds limit, creating new cluster
2025-05-31 17:02:37,089 - DEBUG - Instance 15 exceeds 

In [77]:
rdpm_centroids = rdpm.centroids
labels = rdpm._labels
rdpm_centroids

array([[-3.62326991, -7.7983809 ],
       [ 1.99931164,  3.4609919 ],
       [-9.90777643,  9.33736188],
       [-4.42748779,  9.91766874],
       [ 7.52117345, -5.56987578],
       [-8.0668831 , -6.3874575 ],
       [-4.15410319, -0.41243503],
       [-0.1598857 , -4.58349257],
       [ 6.11777288,  1.45489947],
       [-6.52366919,  5.45625772]])

In [78]:
rdpm_centroids = rdpm.centroids
df = pd.DataFrame(X, columns=['x', 'y'])
df['label'] = rdpm._labels
fig = plot_dataframe(df, title='RDPM Clustering with Constraints', x='x', y='y', label='label')
for i, centroid in enumerate(rdpm_centroids):
    fig = plot_centroid(fig, centroid, colors=[COLORS[i % len(COLORS)]], marker_size=10, symbol='star', name=f'Centroid {i+1}')
fig.show()